In [2]:
from langchain_community.vectorstores.redis import Redis
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {
    'normalize_embeddings': True,
    'batch_size':32,
}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vector_store = Redis(
    redis_url="redis://localhost:6379",
    embedding=hf,
    index_name="users",
)

C:\Users\shahr\AppData\Local\Temp\ipykernel_10296\97828609.py:16: LangChainDeprecationWarning: The class `Redis` was deprecated in LangChain 0.3.13 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-redis package and should be used instead. To use it run `pip install -U :class:`~langchain-redis` and import as `from :class:`~langchain_redis import RedisVectorStore``.
  vector_store = Redis(


In [8]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"baz": "bar"})
document_2 = Document(page_content="thud", metadata={"bar": "baz"})
document_3 = Document(page_content="i will be deleted :(")
document_4 = Document(page_content="shah hobbies explore technology", metadata={"hobby": "technology enthusiast"})
document_5 = Document(page_content="shah works as QA", metadata={"profession": "QA engineer"})
document_6 = Document(page_content="shah from kuala lumpur malaysia, 33 years old")

documents = [document_1, document_2, document_3,document_4,document_5,document_6]
ids = ["1", "2", "3","4","5","6"]
vector_store.add_documents(documents=documents, ids=ids)

['doc:users:1',
 'doc:users:2',
 'doc:users:3',
 'doc:users:4',
 'doc:users:5',
 'doc:users:6']

In [10]:
results = vector_store.similarity_search(query="shah",k=2)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* shah works as QA [{'id': 'doc:users:5'}]
* shah hobbies explore technology [{'id': 'doc:users:4'}]


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os
# Load list of pdfs from a folder
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]
 
print("Listing available documents ...", docs)

# pick out the Nike doc for this exercise
doc = [doc for doc in docs if "nke" in doc][0]
 
# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=2500, chunk_overlap=0
)
loader = PyPDFLoader(doc, headers = None)
# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)
 
print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

Listing available documents ... ['resources/2022-chevy-colorado-ebrochure.pdf', 'resources/aapl-10k-2023.pdf', 'resources/amzn-10k-2023.pdf', 'resources/generation_basic_rag_test.csv', 'resources/jnj-10k-2023.pdf', 'resources/msft-10k-2023.pdf', 'resources/nke-10k-2023.pdf', 'resources/nvd-10k-2023.pdf', 'resources/propositions.json', 'resources/retrieval_basic_rag_test.csv', 'resources/testset.csv', 'resources/testset_15.csv']
Done preprocessing. Created 211 chunks of the original pdf resources/nke-10k-2023.pdf


In [15]:
chunks[:1]

[Document(metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'resources/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}, page_content="Table of Contents\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE FISCAL YEAR ENDED MAY 31, 2023\nOR\n☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE TRANSITION PERIOD FROM                         TO                         .\nCommission File No. 1-10635\nNIKE, Inc.\n(Ex

In [13]:
#embeddings = hf.embed_documents([chunk.page_content for chunk in chunks])
vector_store.add_documents([chunk.page_content for chunk in chunks])
# Check to make sure we've created enough embeddings, 1 per document chunk
len(vector_store) == len(chunks)

AttributeError: 'str' object has no attribute 'id'